#1. Skip Connection (Residual Connection)

##1-1. 문제
이전 연구들에 따르면 네트워크의 깊이는 성능 향상의 핵심적임.
그런데 네트워크 깊이가 증가하면 훈련 정확도가 떨어짐? -> 왜 성능이 나빠지는가?


마치 깊은 미로에서 출구를 못찾는 현상
#
CNN에서 깊이를 증가시키면 오히려 학습이 더 어려워지고 정확도가 떨어짐 -> 성능 저하 문제, 최적화 문제 발생

##1-2. 해결방법
기존 네트워크가 원하는 함수 학습 --> 대신 잔차(Residual)을 학습 == 기존 입력은 그대로, 필요한 변화(잔차)만 추가

이 때 Skip Connection은 가중치가 없고, 연산량이 증가하지 않음. 입력이 그대로 추가되기 때문에 역전파의 안정성도 증가.

##1-3. 해설
기존 Plain 네트워크에 비해 같은 깊이에서 더 낮은 훈련 오류가 검증.
오히려 깊어질수록 정확도가 계속해서 향상.

###입력 / 출력의 크기가 달라질때는?

zero padding shortcut: 숏컷은 계속해서 항등으로 계산하고 부족한 채널을 0으로 패딩

projection shortcut: 1x1 합성곱을 이용해서 차원을 맞춤.

그럼 projection shortcut은 얼마나 성능 향상에 도움이 될까?

A: 차원이 증가할 때만 제로패딩 숏컷 사용

B: 차원이 증가할 때만 프로젝션 숏컷 사용

C: 전부 프로젝션 숏컷 사용

결과는 C > B > A 순으로 성능이 좋음, 그러나 성능 향상에는 미미함.

이유: A는 제로패딩된 채널이 0으로 채워져있기 때문에 잔차학습이 미적용, C는 프로젝션 숏컷 때문에 파라미터가 늘어나서 일어난 성능 향상

결론: 메모리,시간 따지면 projection 숏컷은 굳이 쓸 필요 없음

### 계산량을 더 줄이고 싶은데?
Bottleneck을 이용해 병목 구조 만들기

2층의 잔차 블록 대신에 3층 병목 블록을 사용해서

채널 감소 - 병목 - 채널 복원

블록 덕분에 깊이도 늘리면서 계산량을 줄일 수 있음

###모델이 초기에 불안정한데?

학습률 워밍업을 통해 너무 높은 학습률을 일시적으로 낮추고, 훈련 오류가 80% 아래로 떨어지면 다시 학습률을 복원

###학습은 잘되는데, 과적합은?
Resnet은 규제 관련 논문이 아님

##1-4. 결론
스킵 커넥션을 사용하는 ResNet은 최적화에 유리하고, 깊은 네트워크를 오류가 커지지 않고 안정적으로 학습 가능.

일반적인 네트워크 구성에서 잔차 블록을 적용 가능.

이미지 분류, 객체 탐지에서 큰 향상을 보임.

#2. SE Net

##2-1. 문제

기존 CNN은 모든 채널을 똑같이 취급 -> 필요한 특징만을 뽑아낼 수 없음

강아지 사진에서 배경인 풀은 불필요

어떤 채널이 중요한지, 채널이 어떤 역할을 하는지, 채널의 상호관계는 어떤지 학습할 필요가 있음

##2-2. 해결방법

그럼 채널에서 공간을 빼고, 채널마다 가중치를 부여하면 됨

##2-3. 해설

Squeeze: 평균 풀링으로 채널의 정보를 하나의 값으로 요약

이러면 채널의 공간 정보는 사라짐

Excitation: 재조정? 여기? 렐루와 시그모이드 함수를 사용해 채널별 중요도 가중치를 생성

Scale: 그리고 생성된 가중치를 기존의 Feature map에다 곱해서 채널별로 중요도가 갈림

SE 블록은 적은 연상량의 증가만으로 기존 CNN 구조에 그대로 삽입하면 성능이 향상

##2-4. 결론

SE 블록은 간단하면서 계산량의 증가가 적고, 어떤 모델에다 붙여도 성능이 향상됨




#3. Depthwise Convolution

##3-1. 문제

기존 CNN은 연산량이 너무 많음 == 필터를 거치면서 공간 연산과 채널 연산을 동시에 함

##3-2. 해결 방법

공간 연산과 채널 연산을 완전히 분리시키자.

##3-3. 해설

각각의 채널마다 필터를 적용하고, 그 채널에서 공간 정보만 추출한다.

왜 Depthwise인가? => 채널 방향 (깊이)로 필터를 적용하기 때문

Pointwise: 분리된 채널들을 다시 결합하고 채널 간의 상호작용을 학습

왜 Pointwise인가? =? 1x1 Conv는 1x1이기 때문에 공간 정보가 없음 -> 깊이 (채널)만 학습하게 됨

##3-4. 결론

Depthwise 연산을 사용한 Xception에서 연산량이 8~9배 감소

#4. CBAM

##4-1. 문제

SENet은 채널의 중요도를 파악함. 그런데 공간의 중요도는 고려하지 못함.

강아지 그림에서 강아지가 왼쪽 구석에 있는지, 꽉 차게 있는지를 구분해야 함

##4-2. 해결 방법

Attention을 도입해서 채널에도 집중하고, 공간에도 집중하자

##4-3. 해설


### 채널 집중

피처 맵에서 SENet처럼 채널마다 평균 풀링 해준다. -> 그런데 최대 풀링은 왜 안함?

그럼 평균 풀링과 최대 풀링을 동시에 적용하고 MLP로 묶어서 쓰자.

### 공간 집중

Pointwise에서 처럼 채널을 압축시킴 -> 여기도 평균 풀링 최대 풀링 동시에 쓰자

### BAM과의 차이

BAM은 Channel attention과 Spatial attention을 병렬로 배치, 각각 계산해서 결과를 합침

CBAM은 그 둘을 직렬로 배치, 채널 계산 후 공간 계산

##4-4. 결론

Senet보다 이미지 인식의 성능 증가 (공간 집중을 포함 시켰기 때문에)
연산량 증가 대비 성능 효율적으로 증가
그러나 무겁고 복잡함